# Medallion Architecture: Bronze → Silver → Gold

This notebook demonstrates the medallion architecture pattern for data processing.

## Learning Objectives
- Understand Bronze, Silver, Gold layers
- Implement data quality checks
- Create business-ready datasets
- Apply data governance principles


## 🥉 Bronze Layer: Raw Data Ingestion


In [ ]:
# Load raw airlines data (Bronze layer)
from pyspark.sql.functions import current_timestamp, lit

# Try to load from different sources
try:
    # First try: Use provided input path parameter
    input_path = "/databricks-datasets/airlines"  # This can be passed as parameter
    df = spark.read.csv(input_path, header=True, inferSchema=True)
    print("✅ Loaded data from databricks-datasets")
except:
    try:
        # Second try: Use sample data from FileStore
        df = spark.read.csv("/FileStore/sample_airlines.csv", header=True, inferSchema=True)
        print("✅ Loaded sample data from FileStore")
    except:
        # Third try: Create sample data
        sample_data = [
            (2001, 1, 1, 848, 8, 923, 3, "AA", 1, "N319AA", "JFK", "LAX", 339, 2475, 14, 8),
            (2001, 1, 1, 850, 10, 1006, 6, "AA", 2, "N319AA", "JFK", "LAX", 336, 2475, 14, 10),
            (2001, 1, 1, 923, 23, 1004, 4, "AA", 3, "N319AA", "JFK", "LAX", 321, 2475, 15, 23),
            (2001, 1, 1, 1007, 7, 1130, 0, "AA", 4, "N319AA", "JFK", "LAX", 323, 2475, 16, 7),
            (2001, 1, 1, 1249, 9, 1518, 8, "AA", 5, "N319AA", "JFK", "LAX", 329, 2475, 20, 49)
        ]
        columns = ["year", "month", "day", "dep_time", "dep_delay", "arr_time", "arr_delay", 
                  "carrier", "flight", "tailnum", "origin", "dest", "air_time", "distance", "hour", "minute"]
        df = spark.createDataFrame(sample_data, columns)
        print("✅ Created sample data")

# Add metadata columns for Bronze layer
bronze_df = df.withColumn("ingestion_timestamp", current_timestamp())
bronze_df = bronze_df.withColumn("data_source", lit("airlines_dataset"))
bronze_df = bronze_df.withColumn("layer", lit("bronze"))

print(f"Bronze Layer - Raw Data:")
print(f"Records: {bronze_df.count()}")
print(f"Columns: {len(bronze_df.columns)}")
bronze_df.show(5)


In [ ]:
# Save Bronze layer
bronze_path = "/delta/airlines_bronze"  # This can be passed as parameter
bronze_df.write.format("delta").mode("overwrite").save(bronze_path)
print(f"✅ Bronze layer saved to: {bronze_path}")


## 🥈 Silver Layer: Cleaned & Validated Data


In [ ]:
# Load Bronze data and clean it (Silver layer)
from pyspark.sql.functions import col, when, isnan, isnull, trim, upper

bronze_df = spark.read.format("delta").load(bronze_path)

print("Data Quality Checks:")
print(f"Total records: {bronze_df.count()}")
print(f"Records with null carrier: {bronze_df.filter(bronze_df.carrier.isNull()).count()}")
print(f"Records with null flight: {bronze_df.filter(bronze_df.flight.isNull()).count()}")

# Clean and validate data
silver_df = (bronze_df.filter(
    col("carrier").isNotNull() & 
    col("flight").isNotNull() &
    col("origin").isNotNull() &
    col("dest").isNotNull()
)
.withColumn("carrier", trim(upper(col("carrier"))))
.withColumn("origin", trim(upper(col("origin"))))
.withColumn("dest", trim(upper(col("dest"))))
.withColumn("layer", lit("silver"))
.withColumn("processing_timestamp", current_timestamp()))

print(f"\nSilver Layer - Cleaned Data:")
print(f"Records after cleaning: {silver_df.count()}")
silver_df.show(5)


In [ ]:
# Save Silver layer
silver_path = "/delta/airlines_silver"  # This can be passed as parameter
silver_df.write.format("delta").mode("overwrite").save(silver_path)
print(f"✅ Silver layer saved to: {silver_path}")


## 🥇 Gold Layer: Business-Ready Aggregations


In [ ]:
# Load Silver data and create business metrics (Gold layer)
from pyspark.sql.functions import count, avg, sum, max, min

silver_df = spark.read.format("delta").load(silver_path)

# Create carrier performance metrics
carrier_metrics = (silver_df.groupBy("carrier").agg(
    count("*").alias("total_flights"),
    avg("dep_delay").alias("avg_departure_delay"),
    avg("arr_delay").alias("avg_arrival_delay"),
    avg("distance").alias("avg_distance")
)
.withColumn("layer", lit("gold"))
.withColumn("aggregation_timestamp", current_timestamp()))

print("Gold Layer - Carrier Metrics:")
carrier_metrics.show()

# Save Gold layer
gold_path = "/delta/airlines_gold"  # This can be passed as parameter
carrier_metrics.write.format("delta").mode("overwrite").save(gold_path)
print(f"✅ Gold layer saved to: {gold_path}")


## 📊 Pipeline Summary


In [ ]:
# Pipeline summary
bronze_count = spark.read.format("delta").load(bronze_path).count()
silver_count = spark.read.format("delta").load(silver_path).count()
gold_count = spark.read.format("delta").load(gold_path).count()

print("📈 Medallion Architecture Summary:")
print(f"🥉 Bronze Layer: {bronze_count:,} raw records")
print(f"🥈 Silver Layer: {silver_count:,} cleaned records")
print(f"🥇 Gold Layer: {gold_count} business metrics")
print(f"📊 Data Quality: {((silver_count/bronze_count)*100):.1f}% records passed validation")

print("\n✅ Medallion Architecture pipeline completed successfully!")
print("🎯 All layers (Bronze → Silver → Gold) processed successfully!")
